In [16]:
import torch
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.vocab import Vocab
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import re
import torch.nn as nn

import random
from torcheval.metrics import MultilabelAccuracy
import torch.nn.functional as F

d:\samokat_tech\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.read_csv('./data/train.csv')

In [3]:
def preprocess_text(sen):
    # Удаление пунктуации и чисел, кроме смайликов
    sentence = re.sub(r'(?<![:;=Xx8])([.,!?()])', r' ', sen) # Удаление пунктуации, не затрагивая смайлики
    sentence = re.sub(r'\b\d+\b', ' ', sentence) # Удаление чисел

    # Удаление лишних пробелов
    sentence = re.sub(r'\s+', ' ', sentence).strip()
    
    return sentence

In [4]:
data['text'] = data['text'].fillna('')

In [5]:
data

,index,assessment,tags,text,trend_id_res0,trend_id_res1,trend_id_res2,trend_id_res3,trend_id_res4,trend_id_res5,...,trend_id_res40,trend_id_res41,trend_id_res42,trend_id_res43,trend_id_res44,trend_id_res45,trend_id_res46,trend_id_res47,trend_id_res48,trend_id_res49
0,5652,6.0,"{ASSORTMENT,PROMOTIONS,DELIVERY}","Маленький выбор товаров, хотелось бы ассортиме...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,18092,4.0,"{ASSORTMENT,PRICE,PRODUCTS_QUALITY,DELIVERY}",Быстро,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,13845,6.0,"{DELIVERY,PROMOTIONS,PRICE,ASSORTMENT,SUPPORT}",Доставка постоянно задерживается,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,25060,6.0,"{PRICE,PROMOTIONS,ASSORTMENT}",Наценка и ассортимент расстраивают,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,15237,5.0,"{ASSORTMENT,PRODUCTS_QUALITY,PROMOTIONS,CATALO...",Доставка просто 👍,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8703,661,6.0,{DELIVERY},пойдет,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8704,1870,6.0,"{PROMOTIONS,PRICE,SUPPORT,PRODUCTS_QUALITY}",Не дают абузить поддержка не возвращает деньги...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8705,22650,2.0,"{DELIVERY,PRODUCTS_QUALITY}","Очень плохая доставка в первую очередь, постоя...",1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
8706,13929,NaN,"{ASSORTMENT,CATALOG_NAVIGATION,PAYMENT,DELIVERY}","Хотелось бы получать более свежие продукты, с ...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
train, temp = train_test_split(data, test_size=0.2, random_state=777)
val, test = train_test_split(temp, test_size=0.5, random_state=777)

In [7]:
print(train.shape)
print(val.shape)
print(test.shape)

(6966, 54)
(871, 54)
(871, 54)


In [10]:

def random_swap(sentence, n=5):
    sentence = sentence.split()
    length = len(sentence)
    for _ in range(n):
        idx1, idx2 = random.randint(0, length - 1), random.randint(0, length - 1)
        sentence[idx1], sentence[idx2] = sentence[idx2], sentence[idx1]
    return ' '.join(sentence)

# Применение аугментации к датасету
new_rows = []
for _ in range(200000):
    idx = random.randint(0, len(train) - 1)
    new_row = data.iloc[idx].copy()
    new_row['text'] = random_swap(new_row['text'])  # Случайная перестановка слов
    new_rows.append(new_row)

# Добавление новых строк в датасет
train = pd.concat([data, pd.DataFrame(new_rows)], ignore_index=True)


In [11]:
print(train.shape)
print(val.shape)
print(test.shape)

(208708, 54)
(871, 54)
(871, 54)


In [12]:

# Использование базового токенизатора
tokenizer = get_tokenizer('spacy', language='ru_core_news_lg')
train_iter = iter(train['text'])
vocab = build_vocab_from_iterator(map(tokenizer, train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

In [20]:
class CommentsDataset(Dataset):
    def __init__(self, dataframe, vocab, max_len=500):
        self.dataframe = dataframe
        self.vocab = vocab
        self.max_len = max_len

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        text = self.dataframe.iloc[idx]['text']
        if not isinstance(text, str):
            text = str(text)
        tokens = [self.vocab[token] for token in tokenizer(text)][:self.max_len]
        labels = self.dataframe.iloc[idx][4:].values.astype(float)
        return torch.tensor(tokens, dtype=torch.long), torch.tensor(labels, dtype=torch.float)


In [21]:
def collate_batch(batch):
    label_list, text_list = [], []
    for _text, _label in batch:
        label_list.append(_label)
        text_list.append(_text)
    text_list = pad_sequence(text_list, batch_first=True, padding_value=0)
    labels = torch.stack(label_list)
    return text_list, labels


train_dataset = CommentsDataset(train, vocab)
val_dataset = CommentsDataset(val, vocab)
test_dataset = CommentsDataset(test, vocab)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, collate_fn=collate_batch)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, collate_fn=collate_batch)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, collate_fn=collate_batch)


In [22]:

class MultiLabelNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, output_dim, hidden_dim, dropout_rate=0.2):
        super(MultiLabelNN, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        # Добавляем LSTM слой
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.dropout = nn.Dropout(dropout_rate)  # Добавляем Dropout
        # Обновляем входные размеры для полносвязного слоя
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, text):
        embedded = self.embedding(text)
        # Пропускаем через LSTM
        lstm_out, (hidden, cell) = self.lstm(embedded)
        # Используем только последнее скрытое состояние
        hidden = hidden[-1,:,:]
        dropped = self.dropout(hidden)  # Применяем Dropout
        return self.fc(dropped)



In [23]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    # Инициализация метрики с выбранным критерием
    train_accuracy_metric = MultilabelAccuracy(criteria="exact_match").to(device)
    val_accuracy_metric = MultilabelAccuracy(criteria="exact_match").to(device)

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        train_accuracy_metric.reset()

        for texts, labels in train_loader:
            texts, labels = texts.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(texts)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

            preds = torch.sigmoid(outputs) >= 0.5
            train_accuracy_metric.update(preds, labels)

        avg_loss = total_loss / len(train_loader)
        train_accuracy = train_accuracy_metric.compute()
        print(f'Epoch {epoch+1}, Train Loss: {avg_loss:.4f}, Train Acc: {train_accuracy:.4f}')

        # Валидация
        model.eval()
        val_loss = 0
        val_accuracy_metric.reset()
        with torch.no_grad():
            for texts, labels in val_loader:
                texts, labels = texts.to(device), labels.to(device)
                outputs = model(texts)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                preds = torch.sigmoid(outputs) >= 0.5
                val_accuracy_metric.update(preds, labels)

        avg_val_loss = val_loss / len(val_loader)
        val_accuracy = val_accuracy_metric.compute()
        print(f'Epoch {epoch+1}, Val Loss: {avg_val_loss:.4f}, Val Acc: {val_accuracy:.4f}')





In [24]:
vocab_size = len(vocab)  # Количество уникальных токенов в вашем словаре
embed_dim = 1000  # Размерность вектора слов
hidden_dim = 128  # Размер скрытого состояния LSTM
output_dim = 50  # Количество выходных меток (классов)

model = MultiLabelNN(vocab_size, embed_dim, output_dim, hidden_dim)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001,weight_decay=1e-5) # 0.001


In [25]:
#100к примеров embed_dim = 1000 hidden_dim = 256  lr=0.01   акураси = 0.58(валид и тест)
#100к примеров embed_dim = 100 hidden_dim = 256  lr=0.01   акураси = 0.63 - 0.61(валид и тест)
#100к примеров embed_dim = 100 hidden_dim = 128  lr=0.01   акураси = 0.62 - 0.63(валид и тест)
#300к примеров embed_dim = 100 hidden_dim = 128  lr=0.01   акураси = 0.94 - 0.89(валид и тест)
#150к примеров embed_dim = 100 hidden_dim = 128  lr=0.01   акураси = 0.94 - 0.93(валид и тест)  без регуляризации(без ЛСТМ)


In [26]:
num_epochs = 10
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs)

Epoch 1, Train Loss: 0.0635, Train Acc: 0.4783
Epoch 1, Val Loss: 0.0471, Val Acc: 0.4891
Epoch 2, Train Loss: 0.0335, Train Acc: 0.6265
Epoch 2, Val Loss: 0.0294, Val Acc: 0.6728
Epoch 3, Train Loss: 0.0207, Train Acc: 0.7512
Epoch 3, Val Loss: 0.0218, Val Acc: 0.7577
Epoch 4, Train Loss: 0.0145, Train Acc: 0.8184
Epoch 4, Val Loss: 0.0178, Val Acc: 0.8106
Epoch 5, Train Loss: 0.0117, Train Acc: 0.8512
Epoch 5, Val Loss: 0.0186, Val Acc: 0.8117
Epoch 6, Train Loss: 0.0100, Train Acc: 0.8695
Epoch 6, Val Loss: 0.0151, Val Acc: 0.8462
Epoch 7, Train Loss: 0.0092, Train Acc: 0.8795
Epoch 7, Val Loss: 0.0175, Val Acc: 0.8289
Epoch 8, Train Loss: 0.0088, Train Acc: 0.8846
Epoch 8, Val Loss: 0.0153, Val Acc: 0.8347
Epoch 9, Train Loss: 0.0081, Train Acc: 0.8928
Epoch 9, Val Loss: 0.0158, Val Acc: 0.8416
Epoch 10, Train Loss: 0.0078, Train Acc: 0.8969
Epoch 10, Val Loss: 0.0143, Val Acc: 0.8530


Видно что начиная с 7 эпохи лос вырос а валид метрика упала(сеть переобучилась)- но оставлю пока как есть,проблема в архитектуре именно ЛСТМ(ее надо более тонко настраивать)

In [27]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [28]:
# Инициализация метрики
test_accuracy_metric = MultilabelAccuracy(threshold=0.5, criteria='exact_match').to(device)

# Переключение модели в режим оценки
model.eval()

# Инициализация переменных для сохранения результатов
test_loss = 0.0
test_accuracy_metric.reset()  # Сброс метрики перед использованием

# Отключение вычисления градиентов
with torch.no_grad():
    for texts, labels in test_loader:
        texts, labels = texts.to(device), labels.to(device)
        outputs = model(texts)
        
        # Вычисление потерь
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        
        # Получение предсказаний и обновление метрики
        preds = torch.sigmoid(outputs) >= 0.5
        test_accuracy_metric.update(preds, labels)

# Вычисление и вывод точности на тестовом наборе
test_accuracy = test_accuracy_metric.compute()
print(f'Test Accuracy: {test_accuracy}')

# Вывод среднего значения потерь на тестовом наборе
avg_test_loss = test_loss / len(test_loader)
print(f'Test Loss: {avg_test_loss}')


Test Accuracy: 0.8289322853088379
Test Loss: 0.01943004354169326


In [24]:
# Сохранение весов модели
torch.save(model.state_dict(), 'model_weights.pth')

# Инициализация модели с правильными параметрами
model = MultiLabelNN(vocab_size, embed_dim, output_dim, hidden_dim)
model.load_state_dict(torch.load('model_weights.pth'))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval() 

MultiLabelNN(
  (embedding): Embedding(11729, 1000, padding_idx=0)
  (lstm): LSTM(1000, 128, batch_first=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc): Linear(in_features=128, out_features=50, bias=True)
)

In [25]:
test_df = pd.read_csv('./data/test.csv')

In [26]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16999 entries, 0 to 16998
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   index       16999 non-null  int64  
 1   assessment  16533 non-null  float64
 2   tags        16967 non-null  object 
 3   text        16997 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 531.3+ KB


In [27]:
# Подготовка данных для предсказания
test_dataset = CommentsDataset(test_df, vocab)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, collate_fn=collate_batch)

In [31]:
def get_binary_predictions(model, loader, test_indices):
    model.eval()
    predictions = []
    with torch.no_grad():
        for texts, _ in loader:  # Метки не нужны, так как это предсказание
            texts = texts.to(device)
            outputs = model(texts)
            preds = torch.sigmoid(outputs) >= 0.8  # Применяем порог для получения бинарных значений
            predictions.append(preds.cpu().numpy().astype(int))  # Конвертируем в int для бинарного формата
    return np.concatenate(predictions, axis=0), test_indices

# Получаем бинарные предсказания и индексы
binary_predictions, test_indices = get_binary_predictions(model, test_loader, test_df['index'].values)

# Преобразуем предсказания в DataFrame
binary_predictions_df = pd.DataFrame(binary_predictions, columns=[f'trend_id_res{i}' for i in range(output_dim)])
# Добавляем столбец индекса на первое место
binary_predictions_df.insert(0, 'index', test_indices)

In [32]:
binary_predictions_df.to_csv('binary_predictions.csv', index=False)

In [33]:
binary_predictions_df

,index,trend_id_res0,trend_id_res1,trend_id_res2,trend_id_res3,trend_id_res4,trend_id_res5,trend_id_res6,trend_id_res7,trend_id_res8,...,trend_id_res40,trend_id_res41,trend_id_res42,trend_id_res43,trend_id_res44,trend_id_res45,trend_id_res46,trend_id_res47,trend_id_res48,trend_id_res49
0,5905,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3135,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,9285,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4655,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,16778,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16994,6327,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16995,6428,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16996,9890,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16997,530,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Считаю,что решение через собвственную сеть или предобученный трансформер является самым АДЕКВАТНЫМ - это демонстрирует метрика полученная в результате  работы

Но юзать предобученый трансформер у меня не было желания,так как подгрузка чужой обученой модели с дальнейшем ее файтюном не совсем коректное решение(Не вижу в чем будет заключаться смысл?подгон метрики под лидерборд - не решение)

Если убрать слой с ЛСТМ и сделать регуляризацию 

В чём плюс моего решения?

У нас есть готовый шаблон нейросети,который показывает отличные результаты на собственных эмбэдингах(правда их всего 12к -поэтому результат на лидерборде слабый,было бы больше данных ,а имеено больший словарь(как в тетсте) то результат на лидерборде тоже был бы вполне ничего себе)

Мы можем скормить данной сети больше разнообразных отзывов,получить больше эмбденингов и больше семантической связи,предварительно покрутив архитектуру сети,добавить слои,покрутить дропаут - в результате получаем уникальное решение,заточенное под нашу задачу

Я брал тяжелую мистралевскую модель на матрицу 4к эмбдингов (а не 1024 как у многих) и получил метрику на тесте 0.98,но так как мой ПК не тянет такие,не могу показать(можете сами поюзать) - но в чём тогда моя работа заключается? не ясно

Классический МЛ показывает слабые результаты,я отказался от этой идеи

Не просто так сейчас тренд на НЛП (все трансформеры и решения подобных через ЛЛМ модели),так как они показывают лучшие результаты чем классический МЛ

Можно считать что я не справился с подгоном метрики под лидерборд,но считаю что задачу я решил грамотно и подошёл к решению адекватно(я хорошо прокачал навыки в понимание архитектуры собвственой сети и работы с торчем)

Можно было еще сделать обратный перевод,я планировал по такой схеме

Who Knows, [03.03.2024 18:15]
особенно если с ру на английский и обратно в русский
с этого же английского на французкий и обратно в русский
с этого же английского на испанский и обратно в русский

Who Knows, [03.03.2024 18:16]
Я так понимаю данные хорошие рассширяются потому что 
Это разнное семейство языков

Who Knows, [03.03.2024 18:18]
получается 4 группы языков
немецкий славянский нормандский и латинский

В общем вот мой вариант решения(для более точного решения нужно постоянно обновлять базу эмбэдингов,и возиться в архитектуре сети,что в целом не так уж сложно)